# Pre trained network
![imagen](https://www.researchgate.net/publication/336874848/figure/fig1/AS:819325225144320@1572353764073/Illustrations-of-transfer-learning-a-neural-network-is-pretrained-on-ImageNet-and.png)

Estas son las arquitecturas de redes neuronales más utilizadas en la comunidad. Para más detalle sobre el funcionamiento de cada red, consultar el *Hands on Machine Learning for Python*.
* VGG-16
* VGG-19
* Inception V3
* XCeption
* ResNet-50

Estas redes las podemos incorporar entrenadas, o sin entrenar.

## ResNet50V2

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
import cv2

from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input

In [2]:

'''
imagenet es un corpus de imagenes con el que se ha clasificado el ResNet50V2
http://www.image-net.org/

Cargamos toda la red ya entrenada, y la usaremos para predecir
include_top=True --> Para que incluya la fully connected layer.
include_top=False --> Desarrollamos la fully connected layer
ojo el input shape que sea el de las imagenes que introduciremos. Esta limitado a imagenes de tamaño n
classifier_activation se usa si include_top=True
'''

base_model = ResNet50V2(input_shape=(224, 224,3),
                        include_top=True,
                        weights="imagenet",
                        classifier_activation="softmax")

In [3]:
base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

Cargamos algunas imagenes desde local, para ver qué tal funciona la red ResNet50V2 ya entrenada.

In [4]:
import os


def read_data(path):
    X = []

    for file in os.listdir(path):
        image = imread(path + '/' + file)
        smallimage = cv2.resize(image, (224, 224))
        print(path + '/' + file)
        
        X.append(smallimage)

    return np.array(X)
    

x_test = read_data('img')

# Procesar las imagenes tal y como entran en el modelo
x_test = preprocess_input(x_test)
print(x_test.shape)

img/bear-1.jpg
img/cat.8016.jpg
img/cat.8037.jpg
img/dog.11856.jpg
img/dog.11857.jpg
img/horse.jpg
img/karate.jpg
img/pizza.jpg
(8, 224, 224, 3)


In [5]:
preds = base_model.predict(x_test)

# decode the results into a list of tuples (class, description, probability)
decodes = decode_predictions(preds, top=5)

for j in decodes:
    print('####################')
    for i,decode in enumerate(j):
        print('Predicted:\n', decode[1], decode[2])

####################
Predicted:
 brown_bear 0.999445
Predicted:
 chow 0.00054156024
Predicted:
 American_black_bear 8.781006e-06
Predicted:
 ice_bear 1.1076199e-06
Predicted:
 howler_monkey 4.4061818e-07
####################
Predicted:
 Egyptian_cat 0.64985013
Predicted:
 Siamese_cat 0.16787368
Predicted:
 tiger_cat 0.060449515
Predicted:
 lynx 0.024389781
Predicted:
 tabby 0.017880365
####################
Predicted:
 Egyptian_cat 0.8209842
Predicted:
 lynx 0.12679438
Predicted:
 Siamese_cat 0.021530626
Predicted:
 tabby 0.020569604
Predicted:
 tiger_cat 0.007405382
####################
Predicted:
 Rottweiler 0.732909
Predicted:
 Brabancon_griffon 0.085085
Predicted:
 Staffordshire_bullterrier 0.031059418
Predicted:
 EntleBucher 0.023099238
Predicted:
 Doberman 0.019871488
####################
Predicted:
 collie 0.99525696
Predicted:
 Shetland_sheepdog 0.0047429753
Predicted:
 yellow_lady's_slipper 9.142575e-09
Predicted:
 goldfinch 8.13325e-09
Predicted:
 groenendael 7.536617e-09
####

## VGG16
En este caso vamos a importar la red VGG16, que utilizaremos como red preentrenada y completaremos con una fully connected layer. 

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

IM_SIZE=64

TRAIN_PATH = 'C:/Users/Alberto.Romero/Pictures/The Bridge DS/Redes Neuronales2/cats_dogs_img/train'
#TRAIN_PATH = 'C:/Users/Daney/Desktop/dogs&cats/mini_train/train/'
filenames = os.listdir(TRAIN_PATH)
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    categories.append(category)
    
df = pd.DataFrame({
    'filenames': filenames,
    'category': categories
})

train_df, validate_df = train_test_split(df,
                                         test_size=0.20,
                                         random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [7]:
from keras.preprocessing.image import ImageDataGenerator


# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
validation_datagen = ImageDataGenerator(rescale = 1.0/255. )

In [8]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col='filenames',
                                                              y_col='category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


In [9]:
'''

Note that we have 2 ways of writing the same code:

model = VGG19(weights='imagenet',include_top=False)
model.trainable=False
layer1 = Flatten(name='flat')(model)
layer2 = Dense(512, activation='relu', name='fc1')(layer1)
layer3 = Dense(512, activation='relu', name='fc2')(layer2)
layer4 = Dense(10, activation='softmax', name='predictions')(layer3)

which could be rewritten as:

model = VGG19(weights='imagenet',include_top=False)
model.trainable=False
model.add( Flatten(name='flat'))
model.add( Dense(512, activation='relu', name='fc1'))
model.add( Dense(512, activation='relu', name='fc2'))
model.add( Dense(10, activation='softmax', name='predictions'))

'''

"\n\nNote that we have 2 ways of writing the same code:\n\nmodel = VGG19(weights='imagenet',include_top=False)\nmodel.trainable=False\nlayer1 = Flatten(name='flat')(model)\nlayer2 = Dense(512, activation='relu', name='fc1')(layer1)\nlayer3 = Dense(512, activation='relu', name='fc2')(layer2)\nlayer4 = Dense(10, activation='softmax', name='predictions')(layer3)\n\nwhich could be rewritten as:\n\nmodel = VGG19(weights='imagenet',include_top=False)\nmodel.trainable=False\nmodel.add( Flatten(name='flat'))\nmodel.add( Dense(512, activation='relu', name='fc1'))\nmodel.add( Dense(512, activation='relu', name='fc2'))\nmodel.add( Dense(10, activation='softmax', name='predictions'))\n\n"

In [10]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (IM_SIZE, IM_SIZE, 3), # Shape of our images
                    include_top = False, # Leave out the last fully connected layer
                    weights = 'imagenet')

# No queremos que entrenen en el fit
for layer in base_model.layers:
    layer.trainable = False
    
##### FULLY CONNECTED LAYER #####
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['acc'])

In [11]:
# The Steps per epoch denote the number of batches to be selected for one epoch
#
# A training step is one gradient update. In one step batch_size many examples are processed.
# An epoch consists of one full cycle through the training data. This is usually many steps.
#
# As an example, if you have 2000 images and use a batch_size of 10, an epoch consists of
# 2000 images / (10 images/step) = 200 steps
#
# If you choose your training image randomly and independent in each step, you normally do not call it "epoch"
#

In [ ]:
'''

batch_size determines the number of samples in each mini batch.
Its maximum is the number of all samples, which makes gradient descent accurate,
the loss will decrease towards the minimum if the learning rate is small enough,
but iterations are slower.
Its minimum is 1, resulting in stochastic gradient descent: Fast but the direction of the gradient step is based only on one example,
the loss may jump around.

batch_size allows to adjust between the two extremes: accurate gradient direction and fast iteration.
Also, the maximum value for batch_size may be limited if your model + data set does not fit into the available (GPU) memory.

steps_per_epoch the number of batch iterations before a training epoch is considered finished.
If you have a training set of fixed size you can ignore it but it may be useful if you have a huge data set
or if you are generating random data augmentations on the fly, i.e.
if your training set has a (generated) infinite size.
If you have the time to go through your whole training data set I recommend to skip this parameter.

validation_steps similar to steps_per_epoch but on the validation data set instead on the training data.
If you have the time to go through your whole validation data set I recommend to skip this parameter.

'''

In [12]:
vgghist = model.fit(train_generator,
                    validation_data = validation_generator,
                    steps_per_epoch = 100,
                    epochs = 10)

Epoch 1/10
100/100 [==============================] - 139s 1s/step - loss: 0.7629 - acc: 0.6017 - val_loss: 0.5269 - val_acc: 0.7388
Epoch 2/10
100/100 [==============================] - 137s 1s/step - loss: 0.5782 - acc: 0.7133 - val_loss: 0.5143 - val_acc: 0.7420
Epoch 3/10
100/100 [==============================] - 137s 1s/step - loss: 0.5883 - acc: 0.6717 - val_loss: 0.4743 - val_acc: 0.7718
Epoch 4/10
100/100 [==============================] - 136s 1s/step - loss: 0.5738 - acc: 0.6962 - val_loss: 0.4761 - val_acc: 0.7650
Epoch 5/10
100/100 [==============================] - 135s 1s/step - loss: 0.5373 - acc: 0.7075 - val_loss: 0.4865 - val_acc: 0.7592
Epoch 6/10
100/100 [==============================] - 147s 1s/step - loss: 0.5130 - acc: 0.7422 - val_loss: 0.4749 - val_acc: 0.7684
Epoch 7/10
100/100 [==============================] - 135s 1s/step - loss: 0.5598 - acc: 0.7126 - val_loss: 0.4625 - val_acc: 0.7848
Epoch 8/10
100/100 [==============================] - 132s 1s/step - 

### Inception

In [13]:
IM_SIZE=150

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col='filenames',
                                                              y_col='category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


In [14]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(input_shape = (IM_SIZE, IM_SIZE, 3),
                         include_top = False,
                         weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

In [15]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

inc_history = model.fit(train_generator,
                      validation_data = validation_generator,
                      steps_per_epoch = 100,
                      epochs = 10)


Epoch 1/10
100/100 [==============================] - 162s 2s/step - loss: 2.6778 - acc: 0.7852 - val_loss: 0.1799 - val_acc: 0.9434
Epoch 2/10
100/100 [==============================] - 155s 2s/step - loss: 0.4198 - acc: 0.8826 - val_loss: 0.1419 - val_acc: 0.9572
Epoch 3/10
100/100 [==============================] - 155s 2s/step - loss: 0.4073 - acc: 0.8810 - val_loss: 0.2837 - val_acc: 0.9274
Epoch 4/10
100/100 [==============================] - 156s 2s/step - loss: 0.3501 - acc: 0.9014 - val_loss: 0.1349 - val_acc: 0.9556
Epoch 5/10
100/100 [==============================] - 169s 2s/step - loss: 0.2749 - acc: 0.9207 - val_loss: 0.2399 - val_acc: 0.9414
Epoch 6/10
100/100 [==============================] - 165s 2s/step - loss: 0.3459 - acc: 0.9018 - val_loss: 0.2332 - val_acc: 0.9366
Epoch 7/10
100/100 [==============================] - 160s 2s/step - loss: 0.2839 - acc: 0.9083 - val_loss: 0.1625 - val_acc: 0.9596
Epoch 8/10
100/100 [==============================] - 166s 2s/step - 

## ResNet50V2 sin entrenar

In [16]:
IM_SIZE=32

base_model = ResNet50V2(input_shape=(IM_SIZE, IM_SIZE,3),
                        include_top=False,
                        classifier_activation="softmax")

In [17]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col='filenames',
                                                              y_col='category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


In [18]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

history = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)


Epoch 1/10
100/100 [==============================] - 124s 1s/step - loss: 0.7932 - acc: 0.5034 - val_loss: 0.6935 - val_acc: 0.5432
Epoch 2/10
100/100 [==============================] - 119s 1s/step - loss: 0.7338 - acc: 0.5313 - val_loss: 0.6785 - val_acc: 0.5760
Epoch 3/10
100/100 [==============================] - 120s 1s/step - loss: 0.7045 - acc: 0.5481 - val_loss: 0.6682 - val_acc: 0.6070
Epoch 4/10
100/100 [==============================] - 119s 1s/step - loss: 0.7034 - acc: 0.5628 - val_loss: 1.1696 - val_acc: 0.5868
Epoch 5/10
100/100 [==============================] - 123s 1s/step - loss: 0.6928 - acc: 0.5655 - val_loss: 0.7042 - val_acc: 0.6100
Epoch 6/10
100/100 [==============================] - 120s 1s/step - loss: 0.7028 - acc: 0.5721 - val_loss: 1.2627 - val_acc: 0.6256
Epoch 7/10
100/100 [==============================] - 118s 1s/step - loss: 0.6733 - acc: 0.5898 - val_loss: 0.8063 - val_acc: 0.6460
Epoch 8/10
100/100 [==============================] - 118s 1s/step - 

In [19]:
history = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)


Epoch 1/10
100/100 [==============================] - 116s 1s/step - loss: 0.6531 - acc: 0.6770 - val_loss: 0.6559 - val_acc: 0.6906
Epoch 2/10
100/100 [==============================] - 117s 1s/step - loss: 0.6704 - acc: 0.6340 - val_loss: 0.7215 - val_acc: 0.6748
Epoch 3/10
100/100 [==============================] - 117s 1s/step - loss: 0.6620 - acc: 0.6315 - val_loss: 0.5994 - val_acc: 0.7116
Epoch 4/10
100/100 [==============================] - 117s 1s/step - loss: 0.6398 - acc: 0.6770 - val_loss: 0.8550 - val_acc: 0.7064
Epoch 5/10
100/100 [==============================] - 121s 1s/step - loss: 0.6283 - acc: 0.6795 - val_loss: 1.5717 - val_acc: 0.6562
Epoch 6/10
100/100 [==============================] - 124s 1s/step - loss: 0.6390 - acc: 0.6635 - val_loss: 1.0811 - val_acc: 0.7168
Epoch 7/10
100/100 [==============================] - 120s 1s/step - loss: 0.6410 - acc: 0.6745 - val_loss: 1.3255 - val_acc: 0.6736
Epoch 8/10
100/100 [==============================] - 119s 1s/step - 